In [31]:
# For data
import networkx as nx
from parse import *
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [32]:
from IPython.core.display import display, HTML
from IPython.display import Javascript
from string import Template
import json

In [33]:
H=nx.read_gml('gml_topo/Bren.gml',  label='id')
H

In [34]:
# Base for measure is Mb/s
mmnt = {'b/s':1e-6, 'kb/s': 1e-3, 'Mb/s': 1, 'Gb/s':1e+3, 'Tb/s':1e+6}

In [35]:
def unit_measure(strr):
    strr = strr.replace(" ", "")
    i = strr.rfind('b/s') + 3
    strr = strr[:i]
    for m in mmnt.keys():
        if m in strr:
            pattern = '{:d}' + m
            k = parse(pattern, strr)
            if k != None:
                return k[0]*mmnt[m]
    return None

In [36]:
bflows = {}
for e in H.edges():
    data = H.get_edge_data(e[1], e[0])
    if data['LinkLabel'] != 'Ethernet':
        bandwidth = unit_measure(data['LinkLabel'])
        step = bandwidth / 100.0
        back_flow = random.randrange(0.0, bandwidth, step)
        bflows[e] = {'BackgroundFlow': back_flow}
        
nx.set_edge_attributes(H, bflows)
# print("New data in network")
# print(H.edges(data = True))

In [37]:
def print_Graph(H):
    nodes = []
    for node in H.nodes():
        nodes.append({'id': node, 'label': node})
    edges = []
    for edge in H.edges():
        data = H.get_edge_data(edge[0], edge[1])
        if data['LinkLabel'] != 'Ethernet':
            perc = data['BackgroundFlow'] / unit_measure(data['LinkLabel'])
            color = 'GREEN'
            if perc <= 0.5:
                color = 'GREEN'
            elif perc <= 0.8:
                color = 'YELLOW'
            elif perc <= 1:
                color = 'RED'
            edges.append({'from': edge[1], 'to': edge[0], 
                          'dashes':False, 'label': (str)(unit_measure(data['LinkLabel'])) + 'Mb/s', 
                          'font':{'align': 'top'},
                          'color': color,
                          'width': 4})
        else:
            edges.append({'from': edge[1], 'to': edge[0],
                          'dashes':True, 'label': '0Mb/s', 
                          'font':{'align': 'top'},
                          'color': 'BLACK'})
    return (nodes, edges)

In [38]:
nodes, edges = print_Graph(H)

In [39]:
# Transform the graph into a JSON graph
data = {"nodes":nodes, "edges":edges}
jsonGraph = json.dumps(data, indent=4)

# Send to Javascript
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [40]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('mynetwork');
    
    var options = {
        width: '800px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [130]:
%%html
<div id="mynetwork"></div>

In [41]:
Bandwidth = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))
BackFlows = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))

In [42]:
for edge in H.edges(data = True):
    data_edge = H.get_edge_data(edge[0], edge[1])
    if data_edge['LinkLabel'] != 'Ethernet':
        bndw = unit_measure(data_edge['LinkLabel'])
        bflw = data_edge['BackgroundFlow']
        Bandwidth[edge[1]][edge[0]] = bndw
        BackFlows[edge[1]][edge[0]] = bflw

In [43]:
Bandwidth.head(5)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1000,100,0,1000,100,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1000,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
BackFlows.head(5)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,590,86,0,50,19,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,710,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
from itertools import product

def print_answer(correct, answ, V, N, M):
    if correct == "correct":
        for i, j, k, t in product(V, V, N, M):
            if (answ[i][j][k][t] != None) and (answ[i][j][k][t] > 0.0):
                print("answer[{}][{}][{}][{}] = {}".format(i, j, k, t, answ[i][j][k][t]))
    else:
        print(correct)

In [46]:
K = [{'volume': 1000, 'time':  60, 'A': 1, 'B': 21}, 
	 {'volume': 1000, 'time': 100, 'A': 3, 'B': 21}]

In [47]:
V = set(range(Bandwidth.shape[0]))
N = set(range(len(K)))                       # количество контрактов К
e = 20                                       # шаг по времени
T = 0
for k in N:
    T = max(K[k]['time'], T)     # конец отсчета по времени
t0 = 0                           # начало отсчета по времени
M = set(range(T // e))

In [48]:
import ipynb.fs.full.SolvePyomo as spyomo
import ipynb.fs.full.SolveMIP as mip

In [49]:
correct_pyomo, answ_pyomo, time_pyomo = spyomo.evaluate_model(V, N, M, K, T, e, Bandwidth, BackFlows)
print("Total time: ", time_pyomo)
print_answer(correct_pyomo, answ_pyomo, V, N, M)

    solver failure.
Total time:  6.367559909820557
answer[1][3][0][2] = 50.0
answer[3][21][0][2] = 50.0
answer[3][21][1][3] = 50.0


In [53]:
def send_toJS(t, K, H, answ):
    nodes = []
    for node in H.nodes():
        nodes.append({'id': node, 'label': node})
    edges = []
    for edge in H.edges():
        data = H.get_edge_data(edge[1], edge[0])
        if data['LinkLabel'] != 'Ethernet':
            k_flow = 0
            label = ''
            for k in K:
                if answ[edge[0]][edge[1]][k][t] != 0:
                    k_flow = answ[edge[0]][edge[1]][k][t]
                    label = '№' + (str)(k)
            color = 'BLUE'
            if k_flow != 0:
                perc = (k_flow + data['BackgroundFlow']) / unit_measure(data['LinkLabel'])
                if perc <= 0.5:
                    color = 'GREEN'
                elif perc <= 0.8:
                    color = 'YELLOW'
                elif perc <= 1:
                    color = 'RED'        
            edges.append({'from': edge[1], 'to': edge[0], 
                          'dashes':False, 'label': label, 
                          'font':{'align': 'top'},
                          'color': color,
                          'width': 4})
        else:
            edges.append({'from': edge[1], 'to': edge[0],
                          'dashes':True, 'label': '0Mb/s', 
                          'font':{'align': 'top'},
                          'color': 'BLACK'})
            
    # Transform the graph into a JSON graph
    data = {"nodes":nodes, "edges":edges}
    jsonGraph = json.dumps(data, indent=4)

    return jsonGraph

In [54]:
t = 2
jsonGraph = send_toJS(t, N, H, answ_pyomo)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [55]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_pyomo');
    
    var options = {
        width: '800px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [160]:
%%html
<div id="resultnetwork_pyomo"></div>

In [57]:
correct_mip, answ_mip, time_mip = mip.evaluate_model(V, N, M, K, T, e, Bandwidth, BackFlows)
print("Total time: ", time_mip)
print_answer(correct_mip, answ_mip, V, N, M)

Total time:  6.193553686141968
answer[1][3][0][2] = 50.0
answer[3][21][0][2] = 50.0
answer[3][21][1][3] = 50.0


In [62]:
t = 2
jsonGraph = send_toJS(t, N, H, answ_mip)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [66]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_mip');
    
    var options = {
        width: '800px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [65]:
%%html
<div id="resultnetwork_mip"></div>